In [1]:
import pandas as pd 
import pymysql
import seaborn as sns
import numpy as np

In [10]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [72]:
_host = "13.233.13.169"
_db = "flexiloans_aws_prod_db"
_user= "ds_cron_service"
_password="w76kFYZugHxx96Qk"
con = pymysql.connect(host = _host, db = _db, user = _user, passwd = _password)
cursor=con.cursor()
cursor.execute("SET SESSION sql_mode='';")


df1 = pd.read_sql(""" select 
la.code,la.amount,lad.gender,lbd.has_gst_registration,lad.address_pincode,Year(la.inserted_on)-Year(lad.dob) as AGE,lad.cibil_score,
lbd.address_pin_code,lbd.address_ownership_status,lad.address_ownership_status,lbd.business_vintage,lbd.legal_status,la.inserted_on,ltsr.LAST_IP_COMPLETED_ANALYSIS_PENDING_DATE,
case when ltsr.LAST_IP_COMPLETED_ANALYSIS_PENDING_DATE is null then 0 else 1 end as hot_lead,DATEDIFF( ltsr.LAST_IP_COMPLETED_ANALYSIS_PENDING_DATE,la.inserted_on) as HL_TF,b.total_call,
b.Connect_CALL,la.inserted_on,c.First_Connect,datediff(c.First_Connect,la.inserted_on) as First_Contact
from loan_application la,loan_applicant_detail lad,loan_business_detail lbd,loan_finance_detail lfd,loan_tat_status_report ltsr , partner p ,
(select loan_code,count(loan_code) as total_call ,sum(case when call_connection_status = 'CONTACTED'  then 1 else 0 end) as Connect_CALL  from click_to_call 
group by 1 
) b
,(select loan_code, min(call_start_time)as First_Connect from click_to_call where call_connection_status = 'Contacted'
group by 1 ) c 
where la.code=lad.loan_code and lbd.loan_code=la.code and lfd.loan_code=la.code and ltsr.loan_request_code=la.code and la.inserted_on >="2019-10-01" and la.inserted_on <="2019-12-31"
and la.source_of_lead in ('direct','campaign') and p.code=la.partner_code and b.loan_code=la.code and c.loan_code=la.code
and p.name not like "%Paisaba%"  and la.auto_rejected=0 

""", con=con)

# call_tab = pd.read_sql(""" select la.code,la.inserted_on,cc.call_start_time,hour(timediff(cc.call_start_time,la.inserted_on)) as time_diff
# from loan_application la left join click_to_call cc ON cc.loan_code=la.code 
# where la.code in ('5e0b9a4221v8f','5e29803bg7wzb','5e297edbu4e1h',
# '5e297eeb1myor',
# '5e297f411getu','5e297cefbonwx',
# '5e297cfchp0ck',
# '5e297d0ciakok')""", con=con)



#in ('Other','None')

cursor.close()
con.close()




In [73]:
print(df1.shape[0])
HL=df1[(df1['hot_lead']==1)]
NHL=df1[(df1['hot_lead']==0)]
print(HL.shape[0],"HOT LEADS COUNT")
print(NHL.shape[0],"NOT HOT LEADS COUNT")
print(HL.shape[0]+NHL.shape[0])

28606
2039 HOT LEADS COUNT
26567 NOT HOT LEADS COUNT
28606


In [70]:
#HL['cibil_score']=HL['cibil_score'].apply(pd.to_numeric)     

#### DESCRIPTIVE STATS for Hot Lead
HLS=HL.describe()
HLS.transpose()

,count,mean,std,min,25%,50%,75%,max
amount,3649.00000,1288407.78158,1665865.85740,50000.00000,500000.00000,1000000.00000,1200000.00000,15000000.00000
AGE,3648.00000,36.85691,8.74166,20.00000,30.00000,35.00000,42.00000,75.00000
hot_lead,3649.00000,1.00000,0.00000,1.00000,1.00000,1.00000,1.00000,1.00000
HL_TF,3649.00000,9.55084,13.32878,0.00000,2.00000,4.00000,12.00000,108.00000
total_call,3649.00000,10.27350,12.99465,1.00000,3.00000,6.00000,12.00000,166.00000
Connect_CALL,3649.00000,3.82817,3.78519,1.00000,1.00000,3.00000,5.00000,40.00000
First_Contact,3649.00000,3.17978,7.68486,0.00000,0.00000,1.00000,2.00000,87.00000


In [49]:
NHL['cibil_score']=NHL['cibil_score'].apply(pd.to_numeric)     
NHLS=NHL.describe()
NHLS.transpose()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,count,mean,std,min,25%,50%,75%,max
amount,25883.00000,1297500.02961,2047912.62888,0.00000,500000.00000,500000.00000,1000000.00000,20000000.00000
AGE,23696.00000,34.63964,16.84138,0.00000,28.00000,33.00000,40.00000,1825.00000
cibil_score,2344.00000,659.58916,151.53495,1.00000,656.00000,694.00000,715.00000,872.00000
hot_lead,25883.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
HL_TF,0.00000,nan,nan,nan,nan,nan,nan,nan
total_call,25883.00000,9.67106,16.21979,1.00000,1.00000,3.00000,11.00000,197.00000
Connect_CALL,25883.00000,2.28351,2.43217,1.00000,1.00000,1.00000,3.00000,40.00000
First_Contact,25883.00000,2.78461,6.66800,0.00000,0.00000,1.00000,2.00000,105.00000


In [45]:
HL['HL_TF'].mean()

14.490695396669931